# 教師なし学習実践論（寺口担当分）

注）この資料は、滋賀大学データサイエンス研究科の教師なし学習の教材です。第三者に公開することのないようお願いします。

# Pythonについて

ここでは、プロットにmatplotlibの上に構築された可視化ライブラリであるseaborn（aliasはsns）を使っています。日本語の表示のためにjapanize_matplotlibを読み込んでいます。sklearnはscikit-learn, umapはumap-learnというパッケージ名でpipやcondaでインストールしてください。

df という変数に、全て数値データであるデータフレームが格納されているという前提です。 データをダウンロードして読み込んだだけでは、不必要な行や列が含まれていると思います。不要な部分は最初に除去してください。 データに欠損値がある場合、事前に適当な値で補完しておいてください。

関数の引数はよく使うものだけ記載しています。詳細はヘルプを参照してください。

In [ ]:
# !pip install japanize-matplotlib
# !pip install umap-learn

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import japanize_matplotlib
import sklearn.decomposition
import sklearn.preprocessing
import sklearn.manifold
import umap
import numpy as np

In [ ]:
# 高精細表示設定
%config InlineBackend.figure_format = 'retina'
# 表示をJupyter Notebookに適した設定に変更。ただし、ヒートマップのためにフォントサイズは小さめに設定。
sns.set_context("notebook", font_scale=0.7) 

In [ ]:
df = pd.read_csv("SSDSE-C-2023_formatted.csv",index_col=0)

# ヒートマップ

**`seaborn.clustermap`**`(data, method='average', metric='euclidean', z_score=None, standard_scale=None, row_cluster=True, col_cluster=True)`

**注意点**

-   デフォルトで行と列に対して、階層的クラスタリングが行われる

    -   クラスタリングが不要なら、row_cluster, col_clusterにFalseを指定

    -   method, metricをでクラスタリングのアルゴリズムと距離を変更できる。

-   デフォルトでは正規化が行われない。

    -   z-scoreで行あるいは列に対して正規化するなら、z_score=0あるいはz_score=1

    -   min-max normalizationで行あるいは列に対して正規化するなら、standard_scale=0あるいはstandard_scale=1

-   クラスタリングの機能のない、`seaborn.heatmap()`という関数もある。

In [ ]:
## 生データをプロット
# デフォルトで、行と列のそれぞれがクラスタリングされる。
# 正規化は行われない
sns.clustermap(df)

In [ ]:
# 行に対して正規化（z_scoreは正規化する方向を指定）
# 列（特徴量）の違いが強調される
sns.clustermap(df, z_score=0)

In [ ]:
# 列に対する正規化
# 行（都道府県ごと）の違いが強調される
sns.clustermap(df, z_score=1)

In [ ]:
# 列に対する正規化
# 行に対するクラスタリングは行わない
sns.clustermap(df, z_score=1,row_cluster=False)

In [ ]:
# 列に対する正規化
# クラスタリングは行わない
sns.clustermap(df, z_score=1,row_cluster=False, col_cluster=False)

## 相関行列

`df.corr(axis)`

**注意点**

-   pandasのメソッドで相関行列が計算できる。

-   デフォルトでは、列に対して相関係数が計算される。行に対する相関を計算したければ、df.Tで転置をとった後corr()を呼び出せばよい。

-   カラーマップを調整する（`cmap="bwr", vmin=-1, vmax=1`）ことで、より分かりやすくすることができることもあるが、全てが強く相関しているデータだと逆効果になることも。

In [ ]:
## 列に対する相関行列をプロット
sns.clustermap(df.corr())
sns.clustermap(df.corr(),cmap="bwr", vmin=-1,vmax=1)

In [ ]:
## 行に対する相関行列をプロット
sns.clustermap(df.T.corr())
sns.clustermap(df.T.corr(),cmap="bwr", vmin=-1,vmax=1)

# 次元縮約

## 主成分分析

`sklearn.decomposition.PCA(n_components)`

- n_components：主成分分析による縮約後の次元を指定。特に指定しなければ全て残す。

**注意点**
- scikit-learnのインターフェースなので、クラスをインスタンス化した後で、fit_transform()を呼び出す。
    - 主成分得点はfit_transform()の返り値
    - その他のデータはPCAクラスのインスタンスから取得
- デフォルトではデータの正規化は行われない。（実際には明確な理由がなければ正規化を行った方がよいことが多い。） 
- バイプロットをsklearnで手軽にやる方法は知らないので割愛します。（詳しい人がいれば教えてください。）


In [ ]:
# 正規化を行わずに主成分分析
pca = sklearn.decomposition.PCA()
res = pca.fit_transform(df)

# 図のサイズを変更
#plt.rcParams["figure.figsize"] = (3,3)

# 第１、第２主成分プロット
sns.scatterplot(x=res[:,0],y=res[:,1])
plt.xlabel("PC1")
plt.ylabel("PC2")

In [ ]:
# 正規化を行なって主成分分析
pca = sklearn.decomposition.PCA()
scalar = sklearn.preprocessing.StandardScaler()
res = pca.fit_transform(scalar.fit_transform(df))

# 第１、第２主成分プロット
sns.scatterplot(x=res[:,0],y=res[:,1])
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.show()

In [ ]:
# 第１、第２主成分を行名でプロット
# 最初にs=0でプロットを行うと、点は表示されず、軸のみ描画される。
# その後、text関数で主成分得点の位置に都道府県名（行名）を表示している。
sns.scatterplot(x=res[:,0],y=res[:,1],s=0)
for i in range(0,len(df)):
    plt.text(res[i,0],res[i,1],df.index[i])
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.show()

# 第１、第３主成分を行名でプロット
sns.scatterplot(x=res[:,0],y=res[:,2],s=0)
for i in range(0,len(df)):
    plt.text(res[i,0],res[i,2],df.index[i])
plt.xlabel("PC1")
plt.ylabel("PC3")
plt.show()

In [ ]:
rot = pd.DataFrame(pca.components_,columns=df.columns)
weight = rot.iloc[0].sort_values()
weight_top = pd.concat([weight[:10],weight[-10:]])
sns.barplot(x=weight_top.index,y=weight_top)
plt.xticks(rotation=90)
plt.title("第１主成分の重みベクトル")
plt.show()

In [ ]:
rot = pd.DataFrame(pca.components_,columns=df.columns)
weight = rot.iloc[1].sort_values()
weight_top = pd.concat([weight[:10],weight[-10:]])
sns.barplot(x=weight_top.index,y=weight_top)
plt.xticks(rotation=90)
plt.title("第２主成分の重みベクトル")
plt.show()

In [ ]:
rot = pd.DataFrame(pca.components_,columns=df.columns)
weight = rot.iloc[2].sort_values()
weight_top = pd.concat([weight[:10],weight[-10:]])
sns.barplot(x=weight_top.index,y=weight_top)
plt.xticks(rotation=90)
plt.title("第３主成分の重みベクトル")
plt.show()

In [ ]:
plt.bar(range(pca.n_components_), pca.explained_variance_ratio_)
plt.title("寄与率")
plt.show()

In [ ]:
cumulative_variance_ratio = np.cumsum(pca.explained_variance_ratio_)
plt.title("累積寄与率")
plt.bar(range(pca.n_components_), cumulative_variance_ratio)

ちなみに、主成分得点 res\$xの分散共分散行列を計算すると、確かに対角的になっている。

In [ ]:
import numpy as np
sns.clustermap(np.cov(res.T), row_cluster=False, col_cluster=False)

## t-SNE

`sklearn.manifold.TSNE(n_components=2, perplexity=30.0, metric='euclidean', random_state=None, angle=0.5)`

-   n_components：可視化目的であれば、２次元で十分だが別用途の場合には他の次元にも変更できる
- perplexity：t-SNEのハイパーパラメータ。これにより結果は大きく変わる。ヘルプにはperplexityはデータの行数を超えることのないようにしろと書いてある。
- metric：t-SNEの入力となる距離行列の計算法を選択
- random_state：乱数のシードを与えることができる
-   angle：TSNEでは近似的なt-SNEが利用されている。angleを大きくするほど、スピードが上がるが、本来のt-SNEの結果とはずれていくらしい。





In [ ]:
#データを正規化してからt-SNE
tsne = sklearn.manifold.TSNE(random_state=1234, perplexity=15,angle=0)
res = tsne.fit_transform(scalar.fit_transform(df))

sns.scatterplot(x=res[:,0],y=res[:,1])
plt.xlabel("t-SNE1")
plt.ylabel("t-SNE2")
plt.show()

In [ ]:
# 地域ごとに色指定
cols=np.array([0]+[1]*6+[2]*7+[3]*9+[4]*7+[5]*5+[6]*4+[7]*8)/7
sns.scatterplot(x=res[:,0],y=res[:,1],s=0)
for i in range(0,len(df)):
    plt.text(res[i,0],res[i,1],df.index[i],color=plt.cm.viridis(cols[i]))
plt.xlabel("t-SNE1")
plt.ylabel("t-SNE2")
plt.show()

## UMAP

`umap.UMAP(n_neighbors=15,n_components=2, metric='euclidean', random_state=None)`

- n_neighbors：UMAPのハイパーパラメータ。これにより結果は変わる。
- n_components：可視化目的であれば、２次元で十分だが別用途の場合には他の次元にも変更できる
- metric：UMAPの入力となる距離行列の計算法を選択
- random_state：乱数のシードを与えることができる


In [ ]:
#データを正規化せずにUMAP
u = umap.UMAP(random_state=1234)
res = u.fit_transform(df)

# 地域ごとに色指定
cols=np.array([0]+[1]*6+[2]*7+[3]*9+[4]*7+[5]*5+[6]*4+[7]*8)/7
sns.scatterplot(x=res[:,0],y=res[:,1],s=0)
for i in range(0,len(df)):
    plt.text(res[i,0],res[i,1],df.index[i],color=plt.cm.viridis(cols[i]))
plt.xlabel("UMAP1")
plt.ylabel("UMAP2")
plt.show()

In [ ]:
#データを正規化してからUMAP
u = umap.UMAP(random_state=1234)
res = u.fit_transform(scalar.fit_transform(df))

# 地域ごとに色指定
cols=np.array([0]+[1]*6+[2]*7+[3]*9+[4]*7+[5]*5+[6]*4+[7]*8)/7
sns.scatterplot(x=res[:,0],y=res[:,1],s=0)
for i in range(0,len(df)):
    plt.text(res[i,0],res[i,1],df.index[i],color=plt.cm.viridis(cols[i]))
plt.xlabel("UMAP1")
plt.ylabel("UMAP2")
plt.show()